In [65]:
import pandas as pd
import numpy as np

In [66]:
from sklearn.metrics import mean_squared_error

In [67]:
import os

In [68]:
y_true = pd.read_csv('datasets/test_set.csv')['cpi_pct'].to_numpy()

In [69]:
prediction = []
for file_name in os.listdir('predictions/'):
    
    if file_name[:3] != 'pca':
        continue
    
    y_pred = pd.read_csv(f'predictions/{file_name}', index_col=0).to_numpy()
    
    name = file_name.rsplit('.', 1)[0]
    rmse =  mean_squared_error(y_true, y_pred, squared=False)
    print(name, "rmse:", round(rmse, 7))
    prediction.append({'model': name, 'rmse': rmse})

pca_bma rmse: 0.0030218
pca_lasso_bic rmse: 0.0024414
pca_lasso_cv rmse: 0.0024375
pca_lasso_cvshuffled rmse: 0.0032184
pca_lasso_ebic rmse: 0.0030582
pca_ols rmse: 0.0030814
pca_ridge_cv rmse: 0.0024442


In [70]:
sorting = ['benchmark1_mean', 'benchmark2_prev', 'benchmark3_rf', 
           'ols','ridge_cv', 'lasso_cv', 'lasso_cvshuffled',
           'lasso_bic', 'lasso_ebic', 'bma']

In [71]:
score_df = pd.DataFrame(prediction).set_index('model')

In [72]:
score_df['diff to best'] =  score_df.rmse - score_df.rmse.min()

In [73]:
#score_df.loc[sorting, :].to_csv('results/1970_2023_model_scores.csv')
score_df.sort_values('rmse').to_csv('results/1970_2023_pca_model_scores_sorted.csv')

In [74]:
score_df

,rmse,diff to best
model,,
pca_bma,0.003022,0.000584
pca_lasso_bic,0.002441,0.000004
pca_lasso_cv,0.002438,0.000000
pca_lasso_cvshuffled,0.003218,0.000781
pca_lasso_ebic,0.003058,0.000621
pca_ols,0.003081,0.000644
pca_ridge_cv,0.002444,0.000007


2000-2023 model:

- benchmark1_mean rmse: 0.0026948
- benchmark2_prev rmse: 0.0034082
- bma rmse: 0.002297
- lasso_bic rmse: 0.0023243
- lasso_ebic rmse: 0.0023711
- lasso_cv rmse: 0.0027338
- lasso_cvshuffled rmse: 0.0027338
- ols rmse: 0.0279417
- ridge_cv rmse: 0.0024787

1970-2023 model:

- benchmark1_mean rmse: 0.0032184
- benchmark2_prev rmse: 0.0026672
- benchmark3_rf rmse: 0.0020702
- bma rmse: 0.0020429
- lasso_bic rmse: 0.0021015
- lasso_cv rmse: 0.0021016
- lasso_cvshuffled rmse: 0.0019987
- lasso_ebic rmse: 0.0021058
- ols rmse: 0.0030478
- ridge_cv rmse: 0.0021375

1970-2023 model (PCA):

- benchmark1_mean rmse: 0.0026948
- benchmark2_prev rmse: 0.0034082
- benchmark3_rf rmse: 0.0020702
- bma rmse: 0.002297
- lasso_bic rmse: 0.0023243
- lasso_ebic rmse: 0.0023711
- lasso_cv rmse: 0.0027338
- lasso_cvshuffled rmse: 0.0027338
- ols rmse: 0.0279417
- ridge_cv rmse: 0.0024787

In [112]:
df_list = []


for file_name in os.listdir('models/coefs/'):
    
    coefs = pd.read_csv(f'models/coefs/{file_name}', index_col=0)
    
    if file_name == "bma_coefs.csv":  
        coefs = coefs.rename({'intercept': 'Intercept'})
     
    name = file_name.rsplit('.', 1)[0]
    
    if name[-5:] != 'coefs':
        continue

    coefs.columns = [name[:-6]]
    

    print(name)
    df_list.append(coefs)


bma_coefs
lasso_bic_coefs
lasso_cvshuffled_coefs
lasso_cv_coefs
lasso_ebic_coefs
ols_coefs
ridge_cv_coefs


In [113]:
df = pd.concat(df_list, axis=1).replace(-0, 0)

In [114]:
df.to_csv('results/all_model_coefs.csv')

In [115]:
sorted_idx = df.abs().sort_values('bma', ascending=False).index

In [116]:
df.loc[sorted_idx, :].to_csv('results/all_model_coefs_sorted.csv')